# Creating the time series for the 600+ activities

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
from scipy.sparse import csr_matrix


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
from datetime import datetime

In [3]:
# read files.
with open('../../data/02_intermediate/boston_stays.pickle', 'rb') as f:
    df_boston = pickle.load(f)

# read files.
with open('../../data/02_intermediate/chicago_stays.pickle', 'rb') as f:
    df_chicago = pickle.load(f)

# read files.
with open('../../data/02_intermediate/miami_stays.pickle', 'rb') as f:
    df_miami = pickle.load(f)
    

In [4]:
# save them to a dic
df_dic = {}
df_dic['boston']=df_boston
df_dic['chicago']=df_chicago
df_dic['miami']=df_miami


In [5]:
df_dic['boston'].head()

,user,duration,ini_dat,lon_medoid,lat_medoid,GEOID,home_lon_med,home_lat_med,GEOID_home,quant,fsq_id,cat,distPOI,disthome,hour_of_day,day_of_week,weekday,weekend
0,0000ff45f7f170db960e4e601167975f7559c5be147d69...,388,1.481736e+09,-71.258500,42.359965,25017368400,-71.284748,42.558568,25017316400,3,5390bb9c498e21833a58c208,Office,0.001016,22.188364,14,3,True,False
1,000277100d5593fec35a151e228f6a485210a3fa87cda7...,600,1.478995e+09,-71.149170,42.332710,25021401100,-71.175020,42.300842,25017373900,4,4e5e3cb31838f7255272d5e6,Arts & Entertainment,0.001723,4.132097,20,6,False,True
2,000277100d5593fec35a151e228f6a485210a3fa87cda7...,4086,1.481411e+09,-71.151375,42.336340,25025000502,-71.175020,42.300842,25017373900,4,4e0b572db61cf2466a0f3c6e,Pool,0.000166,4.399966,20,6,False,True
3,000277100d5593fec35a151e228f6a485210a3fa87cda7...,1211,1.481563e+09,-71.207125,42.349600,25017373400,-71.175020,42.300842,25017373900,4,51e1752f498ea376f1942680,Jewelry,0.000149,6.029946,14,1,True,False
4,000277100d5593fec35a151e228f6a485210a3fa87cda7...,621,1.482095e+09,-71.175305,42.318550,25017373900,-71.175020,42.300842,25017373900,4,52edbf4f498e284de9956efb,Supermarket,0.000012,1.969160,17,0,False,True


In [6]:
# find the starting and ending time
# The following time stamp represents 2016/09/30 and 2017/04/02
print(int(min(df_dic['boston'].ini_dat)), int(max(df_dic['boston'].ini_dat)))

1475210333 1491091786


In [7]:
# Use the absolute time stamp and create the daily aggregation
daily_interval = 24 * 60 * 60
starting_time = 1475208000 # EST - 9/30/2016
ending_time = 1491192000 # EST - 4/03/2017

In [11]:
# Create the time series activities. (This line takes about 1.5 hours)
time_series_activities = {}

for city in ['boston','chicago','miami']:
    print()
    print(city)
    time_series_activities[city]={}
    
    # create a list of activities
    activity_dic = {}
    activity_dic[city] = np.unique(df_dic[city].cat, return_counts=False)
    activity_dic[city]

    for activity in activity_dic[city]:
        print(activity)
        daily_count_list = [] # init the daily count list
        activity_df = df_dic[city].loc[df_dic[city].cat==activity, :]
        running_time = copy.copy(starting_time)

        while running_time < ending_time:
            count = np.sum(np.logical_and(activity_df.ini_dat > running_time, activity_df.ini_dat < running_time + daily_interval)) 
            daily_count_list.append(count)
            running_time += daily_interval

        time_series_activities[city][activity] = daily_count_list



boston
ATM
Academic Building
Accessories
Acupuncturist
Administrative Building
Adult Boutique
Adult Education Center
Advertising Agency
Afghan
African
Airport
Airport Service
Alternative Healer
American
Amphitheater
Animal Shelter
Antiques
Apparel
Apres Ski Bar
Aquarium
Arcade
Arepas
Argentinian
Art Gallery
Art Museum
Arts
Arts & Crafts
Arts & Entertainment
Asian
Assisted Living
Astrologer
Athletics & Sports
Auditorium
Australian
Auto Dealer
Auto Garage
Auto Workshop
Automotive
B & B
BBQ
Baby Store
Badminton Court
Bagels
Baggage Claim
Baggage Locker
Bakery
Ballroom
Bank
Bar
Baseball
Baseball Field
Basketball
Basketball Court
Bath House
Bathing Area
Beach
Beach Bar
Beer Bar
Beer Garden
Beer Store
Belgian
Big Box Store
Bike
Bike Shop
Bike Trail
Billiards
Bistro
Board Shop
Boarding
Boat / Ferry
Boat Rental
Bookstore
Border Crossing
Botanical Garden
Boutique
Bowling Alley
Bowling Green
Boxing Gym
Brazilian
Breakfast
Brewery
Bridal
Bridge
Bubble Tea
Buddhist Temple
Buffet
Building
Burgers


Café
Cajun / Creole
Camera Store
Campaign
Campground
Canal
Candy Store
Cantonese
Capitol Building
Car Washes
Caribbean
Carpet Store
Casino
Castle
Cave
Cemetery
Chaat
Champagne Bar
Check Cashing Service
Cheese Shop
Child Care
Chinese
Chiropractors
Chocolate Shop
Christmas Market
Church
Cineplex
Circus
Circus School
City
City Hall
Classroom
Climbing Gym
Club
Cocktail
Coffee Shop
College & Education
College Bookstore
Colombian
Comedy Club
Comfort Food
Comic Shop
Communications
Community Center
Community College
Concert Hall
Conference
Conference room
Construction
Convenience Store
Convention
Convention Center
Corporate Amenity
Corporate Cafeteria
Corporate Coffee Shop
Cosmetics
Costume Shop
Country Dance Club
County
Courthouse
Coworking Space
Credit Union
Creperie
Cruise
Cuban
Cultural Center
Cupcakes
Curling Ice
Currency Exchange
Cycle Studio
Czech
Dance Studio
Daycare
Deli / Bodega
Dentist's Office
Department Store
Design
Desserts
Dim Sum
Diner
Disc Golf
Discount Store
Dispensary
Distil

Currency Exchange
Cycle Studio
Dance Studio
Daycare
Deli / Bodega
Dentist's Office
Department Store
Design
Desserts
Dim Sum
Diner
Disc Golf
Discount Store
Dispensary
Distillery
Distributor
Dive Bar
Dive Shop
Dive Spot
Doctor's Office
Dog Run
Donuts
Driving School
Drugstore
Dry Cleaner
Dumplings
Duty-free
EV Charging
Eastern European
Education
Electronics
Elementary School
Embassy
Emergency Room
Empanada
Engineering
English
Entertainment
Entertainment Service
Erotic Museum
Event Services
Event Space
Exhibit
Eye Doctor
Fabric Shop
Factory
Fair
Falafel
Farm
Farmer's Market
Fast Food
Festival
Field
Filipino
Financial / Legal
Fire Station
Fireworks Store
Fish & Chips
Fish Market
Fishing Spot
Fishing Store
Flea Market
Flight School
Flower Shop
Food
Food & Drink
Food Court
Food Service
Food Stand
Food Truck
Football
Forest
Fountain
Frame Store
Frat House
French
Fried Chicken
Fruit & Vegetable Store
Funeral Home
Furniture / Home
Gaming Cafe
Garden
Garden Center
Gas Station
Gastropub
Gate
Gay B

In [12]:
time_series_activities['miami']['ATM']

[2,
 22,
 18,
 27,
 27,
 32,
 20,
 21,
 23,
 14,
 14,
 17,
 15,
 16,
 10,
 17,
 7,
 17,
 19,
 14,
 18,
 17,
 25,
 17,
 36,
 15,
 28,
 32,
 30,
 18,
 14,
 21,
 20,
 16,
 15,
 36,
 24,
 19,
 12,
 13,
 11,
 14,
 27,
 10,
 14,
 13,
 11,
 24,
 30,
 15,
 15,
 11,
 19,
 15,
 27,
 15,
 11,
 10,
 7,
 6,
 14,
 9,
 8,
 15,
 18,
 12,
 17,
 23,
 29,
 19,
 16,
 13,
 6,
 10,
 10,
 9,
 20,
 18,
 20,
 8,
 16,
 14,
 23,
 28,
 12,
 27,
 8,
 22,
 14,
 27,
 26,
 24,
 19,
 10,
 11,
 11,
 13,
 10,
 15,
 12,
 8,
 22,
 23,
 14,
 11,
 13,
 12,
 24,
 18,
 11,
 13,
 22,
 16,
 16,
 11,
 12,
 11,
 18,
 12,
 16,
 25,
 16,
 28,
 21,
 18,
 21,
 17,
 15,
 6,
 15,
 11,
 11,
 12,
 16,
 25,
 10,
 15,
 16,
 13,
 15,
 14,
 7,
 16,
 20,
 24,
 13,
 5,
 7,
 8,
 12,
 12,
 22,
 13,
 6,
 7,
 11,
 6,
 15,
 20,
 7,
 16,
 13,
 19,
 15,
 19,
 30,
 18,
 10,
 15,
 17,
 12,
 13,
 28,
 11,
 16,
 18,
 9,
 12,
 15,
 7,
 6,
 6,
 8,
 8,
 0]

# Save

In [13]:
with open('../../data/03_processed/temporal_daily_activities.pickle', 'wb') as f:
    pickle.dump(time_series_activities, f)